In [ ]:
import cv2
import numpy as np
from tqdm import tqdm
import os
import sys
import insightface

In [ ]:
synthetic_images = './results/'
embedding_synthetic_path_root = './embeddings/synthetic/'

In [ ]:
fr_model = insightface.app.FaceAnalysis(providers=[
                                        'CUDAExecutionProvider', 'CPUExecutionProvider'])
ctx_id = 0
fr_model.prepare(ctx_id=ctx_id, det_thresh=0.1)

In [ ]:
### Synthetic images
synthetic_face_not_found = np.zeros(0, dtype=object, order='C')
for file in tqdm(os.listdir(synthetic_images)):
    embedding_synthetic_path = embedding_synthetic_path_root + os.path.splitext(file)[0] + ".npy"
    if os.path.exists(embedding_synthetic_path):
        continue
    ext = os.path.splitext(file)[1]
    if (ext == ".jpg" or ext == ".png" or ext == ".jpeg"): # Ignoring other files than .png, jpeg, jpg
        face = fr_model.get(cv2.imread(synthetic_images + file))
        if face:
            embedding = face[0].normed_embedding
            np.save(embedding_synthetic_path,embedding)
        else: # If face is not found, append path of file to list over faces not found
            synthetic_face_not_found = np.append(synthetic_face_not_found, (synthetic_images + file))

In [ ]:
if len(synthetic_face_not_found) != 0:
    print("Number of faces not detected in synthetic images folder:",
          len(synthetic_face_not_found), file=sys.stderr)
    print(synthetic_face_not_found)
    print(*synthetic_face_not_found, sep="\n", file=sys.stderr)